# Browse DICOM Images

This Jupyter Notebook demonstrates an interactive image viewer for browsing DICOM images in axial, sagittal, and coronal views. The notebook utilizes the [XNATpy](https://xnat.readthedocs.io/en/latest/) library along with [pydicom](https://pydicom.github.io/pydicom/stable/) to load DICOM data from XNAT. With the help of XNATpy, you can easily retrieve DICOM files from the XNAT mounts directly on your Jupyter server, and the pydicom library is used for parsing and processing DICOM data.

If XNATpy is not already installed in your environment

In [1]:
!python -c "import xnat" || pip install -q "xnat==0.4.3"

In [2]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import xnat
import time
import os

from ipywidgets import interact
from scipy import ndimage

When connecting to an XNAT server you start by creating a connection. To avoid storing our credentials in our notebook we can use the XNAT environmental variables add to your Jupyter server.

In [3]:
connection = xnat.connect(os.environ['XNAT_HOST'], 
                          user=os.environ['XNAT_USER'], 
                          password=os.environ['XNAT_PASS'])

As of version 0.4.3, XNATpy supports the XNAT/Jupyter envrionment variables so we can shorten this to:

In [4]:
connection = xnat.connect()

Update the code cell below for your particular project, experiment and scan.

In [5]:
session = xnat.connect(loglevel='INFO')
project = session.projects['C4KC-KiTS']
experiment = project.experiments['KiTS-00000_CT_1']
scan = experiment.scans['5']

[INFO] Token login successfully as andrewl
[INFO] GET URI https://xnat.pixi.org/data/JSESSION
[INFO] Determining XNAT version
[INFO] GET URI https://xnat.pixi.org/data/version
[INFO] GET URI https://xnat.pixi.org/xapi/siteConfig/buildInfo?format=json
[INFO] Found an 1.8 version (1.8.8.1)
[INFO] Start parsing schemas and building object model
[INFO] GET URI https://xnat.pixi.org/xapi/schemas?format=json
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/pixi
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/security
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/roi
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/pipeline/repository
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/pipeline/workflow
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/pipeline/build
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/birn/birnprov
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/assessments
[INFO] GET URI https://xnat.pixi.org/xapi/schemas/xdat
[INFO] GET URI https://xnat.pixi.org/xapi/

Next load the DICOM files, sort them by slice location, build a 3d image, and create a widget for each view.

In [6]:
# load the DICOM files
files = []
for file in scan.files.values():
    if file.uri.endswith("dcm"):
        files.append(pydicom.dcmread(file.open()))
        
print("file count: {}".format(len(files)))

# skip files with no SliceLocation (eg scout views)
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

# ensure they are in the correct order
slices = sorted(slices, key=lambda s: s.SliceLocation)

# pixel aspects, assuming all slices are the same
ps = slices[0].PixelSpacing
ss = slices[0].SliceThickness
ax_aspect = ps[1]/ps[0]
sag_aspect = ps[1]/ss
cor_aspect = ss/ps[0]

# create 3D array
img_shape = list(slices[0].pixel_array.shape)
img_shape.append(len(slices))
img3d = np.zeros(img_shape)

# fill 3D array with the images from the files
for i, s in enumerate(slices):
    img2d = s.pixel_array
    img3d[:, :, i] = img2d


def browse_images_ax(img3d, img_shape, ax_aspect):
    n = img_shape[2]
    def view_image(i):
        a = plt.subplot()
        plt.imshow(img3d[:, :, i], cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(ax_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))
    
def browse_images_sag(img3d, img_shape, sag_aspect):
    n = img_shape[1]
    def view_image(i):
        a = plt.subplot()
        rotated_img = ndimage.rotate(img3d[:, i, :], 90)
        plt.imshow(rotated_img, cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(1/sag_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))  
    
def browse_images_cor(img3d, img_shape, cor_aspect):
    n = img_shape[0]
    def view_image(i):
        a = plt.subplot()
        rotated_img = ndimage.rotate(img3d[i, :, :].T, 180)
        plt.imshow(rotated_img, cmap=plt.cm.bone, interpolation='nearest')
        a.set_aspect(cor_aspect)
        plt.title('Image: %s' % i)
        plt.show()
    interact(view_image, i=(0,n-1))  

[INFO] GET URI https://xnat.pixi.org/data/archive/projects/C4KC-KiTS/subjects/PIXI35_S06104/experiments/PIXI35_E06113/scans/5/files?columns=ID%2CURI%2Cpath&format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113/scans/5/resources/7599?format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113/scans/5/resources?format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113/scans/5?format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113?format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113?format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113/scans/5?format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113/scans/5/resources/7599?format=json
[INFO] GET URI https://xnat.pixi.org/data/experiments/PIXI35_E06113/scans/5/resources?format=json
[INFO] Opening file from filesystem!
[INFO] Opening file from filesystem!
[INFO] Opening file 

file count: 93
skipped, no SliceLocation: 0


Notice the log message. XNATpy understands the layout of the XNAT/Jupyter filesystem. If XNATpy can find your data on the filesystem it will use that instead of attempting to download from XNAT.

In [7]:
browse_images_ax(img3d, img_shape, ax_aspect)
browse_images_sag(img3d, img_shape, sag_aspect)
browse_images_cor(img3d, img_shape, cor_aspect)

interactive(children=(IntSlider(value=46, description='i', max=92), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=255, description='i', max=511), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=255, description='i', max=511), Output()), _dom_classes=('widget-interac…